# Продуктовый кейс: воронка и удержание (e-commerce)

## Цель
Понять:
- где пользователи “теряются” до покупки (воронка)
- возвращаются ли они за повторной покупкой (retention)

In [65]:
# импорт библиотек
from pathlib import Path
import pandas as pd
import numpy as np

In [70]:
# загрузка данных
DATA_PATH = Path('../data/events_ecom.csv')
df = pd.read_csv(DATA_PATH, parse_dates=['event_time'])
df.head(10)

,user_id,event_time,event_name,amount,source
0,1,2025-10-11 11:59:00,visit,NaN,ads
1,2,2025-10-07 18:18:00,visit,NaN,organic
2,3,2025-10-20 19:19:00,visit,NaN,referral
3,3,2025-10-21 03:19:00,signup,NaN,referral
4,3,2025-10-23 02:24:00,add_to_cart,NaN,referral
5,4,2025-10-24 23:52:00,visit,NaN,ads
6,5,2025-10-26 05:25:00,visit,NaN,organic
7,6,2025-10-13 11:42:00,visit,NaN,email
8,7,2025-10-03 09:24:00,visit,NaN,ads
9,8,2025-10-07 08:02:00,visit,NaN,ads


## Обзор данных

Зачем:
- понять размер датасета
- увидеть пропуски
- проверить какие есть события и источники
- понять период данных

In [73]:
# количество строк, количество столбцов
df.shape  

(7019, 5)

In [74]:
# проверка структуры и пропусков
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7019 entries, 0 to 7018
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     7019 non-null   int64         
 1   event_time  7019 non-null   datetime64[ns]
 2   event_name  7019 non-null   object        
 3   amount      307 non-null    float64       
 4   source      7019 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 274.3+ KB


user_id          0
event_time       0
event_name       0
amount        6712
source           0
dtype: int64

In [75]:
# сколько раз встречается каждый тип события
df['event_name'].value_counts()

event_name
visit          5000
signup         1192
add_to_cart     520
purchase        307
Name: count, dtype: int64

In [76]:
# распределение по источникам
df["source"].value_counts()

source
organic     3611
ads         2121
referral     677
email        610
Name: count, dtype: int64

In [77]:
# минимальная и максимальная дата/время
df['event_time'].min(), df['event_time'].max()

(Timestamp('2025-10-01 00:01:00'), Timestamp('2025-11-05 15:59:00'))

## Проверка корректности данных (sanity-check): amount должен быть заполнен только для события purchase

In [78]:
# проверяю, есть ли покупки без суммы
purchase_without_amount = df[(df['event_name'] == 'purchase') & (df['amount'].isna())]
purchase_without_amount.head(10)

,user_id,event_time,event_name,amount,source


In [79]:
# проверяю, есть ли amount (сумма) у события отличного от purchase (покупка)
amount_on_non_purchase = df[(df['event_name'] != 'purchase') & (df['amount'].notna())]
amount_on_non_purchase.head(10)

,user_id,event_time,event_name,amount,source


## Таблица шагов на уровне пользователя (user-level)

Зачем:
воронку считаем по пользователям, а не по числу событий

Следовательно, для каждого user_id необходимо найти время первого появления каждого события:
visit, signup, add_to_cart, purchase

Если события не было — NaT

In [85]:
# нахожу время первого шага по каждому событию через pivot
first_step_time = df.pivot_table(
    index='user_id',
    columns='event_name',
    values='event_time',
    aggfunc='min'
)

first_step_time.head(10)

event_name,add_to_cart,purchase,signup,visit
user_id,,,,
1,NaT,NaT,NaT,2025-10-11 11:59:00
2,NaT,NaT,NaT,2025-10-07 18:18:00
3,2025-10-23 02:24:00,NaT,2025-10-21 03:19:00,2025-10-20 19:19:00
4,NaT,NaT,NaT,2025-10-24 23:52:00
5,NaT,NaT,NaT,2025-10-26 05:25:00
6,NaT,NaT,NaT,2025-10-13 11:42:00
7,NaT,NaT,NaT,2025-10-03 09:24:00
8,NaT,NaT,2025-10-07 11:07:00,2025-10-07 08:02:00
9,NaT,NaT,NaT,2025-10-10 00:41:00


## Воронка по пользователям

- сколько пользователей дошло до каждого шага
- конверсию от начала (от visit)
- конверсию к предыдущему шагу

In [91]:
# фиксирую порядок шагов воронки
steps = ["visit", "signup", "add_to_cart", "purchase"]

In [92]:
users_by_step = {step: first_step_time[step].notna().sum() for step in steps}
users_by_step

{'visit': np.int64(5000),
 'signup': np.int64(1192),
 'add_to_cart': np.int64(520),
 'purchase': np.int64(280)}

In [100]:
funnel = pd.DataFrame({
    "step": steps,
    "users": [users_by_step[s] for s in steps]
})

# конверсия от начала
funnel["conv_from_start"] = funnel["users"] / funnel.loc[0, "users"]

# конверсия к предыдущему шагу
# shift(1) сдвигает users на 1 вниз, чтобы signup делился на visit и т.д.
funnel["conv_to_prev"] = funnel["users"] / funnel["users"].shift(1)

funnel

,step,users,conv_from_start,conv_to_prev
0,visit,5000,1.0000,NaN
1,signup,1192,0.2384,0.238400
2,add_to_cart,520,0.1040,0.436242
3,purchase,280,0.0560,0.538462


## Retention 7 и 14 дней

Доли пользователей, которые сделали повторную покупку в течение 7 и 14 дней после первой покупки

In [153]:
# беру только покупки и нужные колонки
p = df.loc[df["event_name"] == "purchase", ["user_id", "event_time"]].copy()
p

,user_id,event_time
29,20,2025-11-02 04:27:00
34,22,2025-10-13 07:39:00
41,25,2025-10-20 04:34:00
54,34,2025-10-16 03:18:00
102,71,2025-10-05 04:09:00
...,...,...
6899,4910,2025-10-08 01:55:00
6900,4910,2025-10-25 01:55:00
6921,4925,2025-10-14 02:03:00
6926,4927,2025-10-25 12:48:00


In [154]:
# добавляю столбец для подсчета разницы в днях
p["d"] = p["event_time"].dt.normalize()

In [155]:
# нахожу дату первой покупки по каждому пользователю
first = p.groupby("user_id")["d"].min()

In [156]:
# добавляю дату первой покупки к каждой покупке пользователя
p = p.merge(first, on="user_id")

In [158]:
# сколько дней прошло от первой покупки до текущей покупки
p["dd"] = (p["d_x"] - p["d_y"]).dt.days

In [159]:
# сколько пользователей вообще купили хотя бы раз
buyers_base = first.size

In [160]:
# сколько пользователей купили повторно в окне 1..7 дней
d7_users = p.loc[p["dd"].between(1, 7), "user_id"].nunique()

In [161]:
# сколько пользователей купили повторно в окне 1..14 дней
d14_users = p.loc[p["dd"].between(1, 14), "user_id"].nunique()

In [162]:
# итоговая таблица
retention_summary = pd.DataFrame({
    "metric": ["buyers_base", "D7_users", "D14_users"],
    "value":  [buyers_base, d7_users, d14_users],
    'retention': [1, d7_users / buyers_base, d14_users / buyers_base]
})

retention_summary

,metric,value,retention
0,buyers_base,280,1.000000
1,D7_users,10,0.035714
2,D14_users,20,0.071429


## Итоговый вывод

### Что было сделано
1) Загрузил события интернет-магазина за октябрь 2025 и проверил данные (размер, типы, пропуски, распределения)
2) Проверили корректность логирования:
   - сумма **amount** заполнена только у **purchase**
   - нет покупок без предыдущих шагов
3) Посчитал воронку **по пользователям** (уникальные пользователи на каждом шаге)
4) Посчитал **retention** по покупкам: доля пользователей, сделавших повторную покупку в окне 7 и 14 дней после первой покупки

### Что получил
**Воронка (по пользователям):**
- visit: **5000** (100%)
- signup: **1192** (**23.8%** от визитов)
- add_to_cart: **520** (**10.4%** от визитов, или **43.6%** от signup)
- purchase: **280** (**5.6%** от визитов, или **53.8%** от add_to_cart)

Главная “утечка” воронки — **переход из визита в регистрацию**

**Retention по покупкам:**
- база покупателей: **280**
- D7 retention: **3.6%** (10 пользователей вернулись с повторной покупкой за 7 дней)
- D14 retention: **7.1%** (20 пользователей вернулись за 14 дней)